In [32]:
#Run all this tools with agents and LLM models
## tools -->agent excutor
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq


In [33]:
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")


In [34]:
llm = ChatGroq(model_name="mistral-saba-24b", temperature=0.7, max_tokens=4096)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7755de13f620>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7755ddf1fcb0>, model_name='mistral-saba-24b', model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=4096)

In [35]:
from langchain.schema import(
    AIMessage,
    HumanMessage,SystemMessage,
    
)

In [36]:
speech="""
People across the country, involved in government, political, and social activities, are dedicating th

In our country, governments have come and gone, numerous schemes have been formulated, discussions hav

It is crucial to check whether the work that was supposed to happen has indeed taken place. I recentl

The bureaucrats, who prepare good schemes, expedite the paperwork and even allocate funds, also feel s

There are very few who understand the power and impact of the [‘Viksit Bharat Sankalp Yatra’].
"""


In [37]:
speech

'\nPeople across the country, involved in government, political, and social activities, are dedicating th\n\nIn our country, governments have come and gone, numerous schemes have been formulated, discussions hav\n\nIt is crucial to check whether the work that was supposed to happen has indeed taken place. I recentl\n\nThe bureaucrats, who prepare good schemes, expedite the paperwork and even allocate funds, also feel s\n\nThere are very few who understand the power and impact of the [‘Viksit Bharat Sankalp Yatra’].\n'

In [38]:
chat_message=[
    SystemMessage(content="You are expert with expertize in summarizing speeched"),
    HumanMessage(content=f"Please provide a short and concisse summary of the follow speech:\n Text{speech}")
]

In [39]:
llm.get_num_tokens(speech)

118

In [40]:
#getting message
llm(chat_message).content

"The speech emphasizes the need to ensure that government initiatives and reforms are effectively implemented. It highlights the frustration among bureaucrats and the public due to the lack of progress in promised schemes. The speaker also notes that the significance of efforts like the 'Viksit Bharat Sankalp Yatra' is often overlooked."

In [41]:
##prompt template text summarization
from langchain.chains import LLMChain
from langchain import PromptTemplate

generictemplate="""
Write a summary of the following speech
Speech:{speech}
Translate the precise summary to {language}
"""

In [42]:
prompt=PromptTemplate(
    input_variables=['sppech','language'],
    template=generictemplate
)
prompt

PromptTemplate(input_variables=['language', 'speech'], input_types={}, partial_variables={}, template='\nWrite a summary of the following speech\nSpeech:{speech}\nTranslate the precise summary to {language}\n')

In [43]:
complete_prompt=prompt.format(speech=speech,language="Hindi")

In [44]:
llm.get_num_tokens(complete_prompt)

139

In [45]:
llm_chain=LLMChain(llm=llm,prompt=prompt)
summary=llm_chain.run({'speech':speech,'language':'Hindi'})
summary

"Certainly! Here is a summary of the speech translated into Hindi:\n\n---\n\nहमारे देश में सरकारें आती-जाती रही हैं, अनेक योजनाएं बनाई गई हैं, चर्चाएं होती रही हैं...\n\nयह जांचना जरूरी है कि वह काम जो होना तय हुआ था, वह सचमुच हुआ है या नहीं। हाल ही में...\n\nवे नौकरशाह जो अच्छी योजनाएं बनाते हैं, काम को तेजी से आगे बढ़ाते हैं और पैसे भी आवंटित करते हैं, वे भी थक गए हैं...\n\nबहुत कम लोग हैं जो 'विकसित भारत संकल्प यात्रा' की शक्ति और प्रभाव को समझते हैं...\n\n---\n\nThis translation captures the essence of the original speech while maintaining its structure and key points."

In [46]:
## stuff document chain
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("APJ-Abdul-Kalam-Suhail-PPT.pdf")
docs=loader.load_and_split()
docs

[Document(metadata={'producer': 'Skia/PDF m92', 'creator': 'PyPDF', 'creationdate': '', 'source': 'APJ-Abdul-Kalam-Suhail-PPT.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1'}, page_content='My Vision For India\nDr. A.P.J. Abdul Kalam\nE-Learning - Department of English\nC. Abdul Hakeem College (A)\nPrepared by\nK. Md. Suhail\nAssistant Professor of English'),
 Document(metadata={'producer': 'Skia/PDF m92', 'creator': 'PyPDF', 'creationdate': '', 'source': 'APJ-Abdul-Kalam-Suhail-PPT.pdf', 'total_pages': 16, 'page': 1, 'page_label': '2'}, page_content='Introduction:\n“Dream,\ndream,\ndream.\nDreams\ntransform\ninto\nthoughts\nand\nthoughts\nresult\nin\naction.”\n–\nA\nP\nJ\nAbdul\nKalam\n\x00 \nAvul\nPakir\nJainulabdeen\nAbdul\nKalam\nwas\nborn\nto\nJainulabdeen,\na\nboat\nowner\nand\nImam,\nand\nAshiamma, a housewife. Kalam had four brothers and a sister, all elder to him.\n\x00 \nHe was born in Rameshwaram in Tamil Nadu.\n\x00 \nHe\nwas\nknown\nas\nthe\n‘Missile\nMan\nof\nIndia’

In [47]:
template="""write a concise and short summary of the following speech,
Speech:{text}
 
"""
prompt=PromptTemplate(input_variables=['text'],
                      template=template)

In [48]:
from langchain.chains.summarize import load_summarize_chain

In [49]:
chain=load_summarize_chain(llm,chain_type='stuff',prompt=prompt,verbose=True)
chain.run(docs)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
write a concise and short summary of the following speech,
Speech:My Vision For India
Dr. A.P.J. Abdul Kalam
E-Learning - Department of English
C. Abdul Hakeem College (A)
Prepared by
K. Md. Suhail
Assistant Professor of English

Introduction:
“Dream,
dream,
dream.
Dreams
transform
into
thoughts
and
thoughts
result
in
action.”
–
A
P
J
Abdul
Kalam
  
Avul
Pakir
Jainulabdeen
Abdul
Kalam
was
born
to
Jainulabdeen,
a
boat
owner
and
Imam,
and
Ashiamma, a housewife. Kalam had four brothers and a sister, all elder to him.
  
He was born in Rameshwaram in Tamil Nadu.
  
He
was
known
as
the
‘Missile
Man
of
India’
for
his
work
in
the
development
of
ballistic
missile
technology.
  
Dr. A.P.J. Abdul Kalam was the President of India between 2002 and 2007.
  
He became the 11
th
 President of India and is one of the most distinguished scientists of India.
  
The essay is one of the most popular s

'Dr. A.P.J. Abdul Kalam\'s speech "My Vision for India" outlines his aspirations for India\'s future, emphasizing three key visions: freedom, development, and global standing. Kalam emphasizes the importance of protecting India\'s independence and transforming it from a developing to a developed nation. He criticizes the media for focusing on negativity and the public for relying overly on foreign imports and blaming the system for their actions. Kalam urges Indians to take responsibility and contribute positively, concluding with a call to action echoing J.F. Kennedy\'s famous words, "Ask what we can do for India."'

In [1]:
from langchain_community.document_loaders import YoutubeLoader
loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=QsYGlZkevEg", add_video_info=False
)
loader.load()

[Document(metadata={'source': 'QsYGlZkevEg'}, page_content='LADIES AND GENTLEMEN, PEDRO PASCAL! [ CHEERS AND APPLAUSE ] >> THANK YOU, THANK YOU. THANK YOU VERY MUCH. I\'M SO EXCITED TO BE HERE. THANK YOU. I SPENT THE LAST YEAR SHOOTING A SHOW CALLED "THE LAST OF US" ON HBO. FOR SOME HBO SHOES, YOU GET TO SHOOT IN A FIVE STAR ITALIAN RESORT SURROUNDED BY BEAUTIFUL PEOPLE, BUT I SAID, NO, THAT\'S TOO EASY. I WANT TO SHOOT IN A FREEZING CANADIAN FOREST WHILE BEING CHASED AROUND BY A GUY WHOSE HEAD LOOKS LIKE A GENITAL WART. IT IS AN HONOR BEING A PART OF THESE HUGE FRANCHISEs LIKE "GAME OF THRONES" AND "STAR WARS," BUT I\'M STILL GETTING USED TO PEOPLE RECOGNIZING ME. THE OTHER DAY, A GUY STOPPED ME ON THE STREET AND SAYS, MY SON LOVES "THE MANDALORIAN" AND THE NEXT THING I KNOW, I\'M FACE TIMING WITH A 6-YEAR-OLD WHO HAS NO IDEA WHO I AM BECAUSE MY CHARACTER WEARS A MASK THE ENTIRE SHOW. THE GUY IS LIKE, DO THE MANDO VOICE, BUT IT\'S LIKE A BEDROOM VOICE. WITHOUT THE MASK, IT JUST SOUNDS

In [50]:
## Map Reduce To Summarize Large Documents

from langchain_text_splitters import RecursiveCharacterTextSplitter
final_doc=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=100).split_documents(docs)
final_doc

[Document(metadata={'producer': 'Skia/PDF m92', 'creator': 'PyPDF', 'creationdate': '', 'source': 'APJ-Abdul-Kalam-Suhail-PPT.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1'}, page_content='My Vision For India\nDr. A.P.J. Abdul Kalam\nE-Learning - Department of English\nC. Abdul Hakeem College (A)\nPrepared by\nK. Md. Suhail\nAssistant Professor of English'),
 Document(metadata={'producer': 'Skia/PDF m92', 'creator': 'PyPDF', 'creationdate': '', 'source': 'APJ-Abdul-Kalam-Suhail-PPT.pdf', 'total_pages': 16, 'page': 1, 'page_label': '2'}, page_content='Introduction:\n“Dream,\ndream,\ndream.\nDreams\ntransform\ninto\nthoughts\nand\nthoughts\nresult\nin\naction.”\n–\nA\nP\nJ\nAbdul\nKalam\n\x00 \nAvul\nPakir\nJainulabdeen\nAbdul\nKalam\nwas\nborn\nto\nJainulabdeen,\na\nboat\nowner\nand\nImam,\nand\nAshiamma, a housewife. Kalam had four brothers and a sister, all elder to him.\n\x00 \nHe was born in Rameshwaram in Tamil Nadu.\n\x00 \nHe\nwas\nknown\nas\nthe\n‘Missile\nMan\nof\nIndia’

In [51]:
len(final_doc)

16

In [52]:
prompt

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='write a concise and short summary of the following speech,\nSpeech:{text}\n\n')

In [53]:
chunk_prompt="""please summarize the below speech,
Speech:'{text}'
Summary:
 
"""
map_prompt_template=PromptTemplate(input_variables=['text'],
                      template=chunk_prompt)

In [54]:
final_prompt='''
Provide the final summary of the entire speech with these important points.
Add a motivation Title ,start the precise summary with an introduction and provide the summary in number
points for the sppech
Speech:{text}
'''
final_prompt_template=PromptTemplate(input_varibles=['Text'],template=final_prompt)
final_prompt_template

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nProvide the final summary of the entire speech with these important points.\nAdd a motivation Title ,start the precise summary with an introduction and provide the summary in number\npoints for the sppech\nSpeech:{text}\n')

In [ ]:
summary_chain=load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt_template,
    combine_prompt=final_prompt_template,
    verbose=True
)
output=summary_chain.run(final_doc)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
please summarize the below speech,
Speech:'My Vision For India
Dr. A.P.J. Abdul Kalam
E-Learning - Department of English
C. Abdul Hakeem College (A)
Prepared by
K. Md. Suhail
Assistant Professor of English'
Summary:


Prompt after formatting:
please summarize the below speech,
Speech:'Introduction:
“Dream,
dream,
dream.
Dreams
transform
into
thoughts
and
thoughts
result
in
action.”
–
A
P
J
Abdul
Kalam
  
Avul
Pakir
Jainulabdeen
Abdul
Kalam
was
born
to
Jainulabdeen,
a
boat
owner
and
Imam,
and
Ashiamma, a housewife. Kalam had four brothers and a sister, all elder to him.
  
He was born in Rameshwaram in Tamil Nadu.
  
He
was
known
as
the
‘Missile
Man
of
India’
for
his
work
in
the
development
of
ballistic
missile
technology.
  
Dr. A.P.J. Abdul Kalam was the President of India between 2002 and 2007.
  
He became the 11
th
 President of India and is one of the most distinguished sc


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Provide the final summary of the entire speech with these important points.
Add a motivation Title ,start the precise summary with an introduction and provide the summary in number
points for the sppech
Speech:Sure, here is a summary of Dr. A.P.J. Abdul Kalam's "My Vision for India" speech:

Dr. A.P.J. Abdul Kalam, in his visionary speech "My Vision for India," outlines his aspirations for the nation's future. He emphasizes the importance of education, innovation, and a strong work ethic in transforming India into a developed country. Kalam envisions India as a nation where every citizen has access to quality education and healthcare, and where technology plays a crucial role in national development. He highlights the need for unity among diverse communities and the importance of ethical values. Kalam also stresses the role of youth in driving change and urges them to take responsibility for building a bett

In [56]:
output

'### **Motivation Title: "Dreaming Towards a Prosperous India: The Vision of Dr. A.P.J. Abdul Kalam"**\n\n**Introduction:**\nDr. A.P.J. Abdul Kalam\'s "My Vision for India" is a profound and inspiring speech that outlines his aspirations for India\'s future. Kalam, a renowned scientist and former President of India, envisions a nation where every citizen has access to quality education, healthcare, and innovation. He emphasizes unity, hard work, and self-reliance as critical drivers for India to become a developed and respected global leader.\n\n**Summary:**\n\n1. **The Power of Dreaming:**\n   - Kalam emphasizes the transformative power of dreaming, which catalyzes thoughts and actions leading to tangible outcomes.\n\n2. **Personal Background:**\n   - Dr. Kalam was born in Rameshwaram, Tamil Nadu, and is known as the ‘Missile Man of India’ for his contributions to ballistic missile technology. He served as India\'s President from 2002 to 2007.\n\n3. **Key Visions for India:**\n   - **

In [60]:
## refine chain

chain=load_summarize_chain(
    llm=llm,
    chain_type='refine',
    verbose=True
)
output_summary=chain.run(final_doc)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"My Vision For India
Dr. A.P.J. Abdul Kalam
E-Learning - Department of English
C. Abdul Hakeem College (A)
Prepared by
K. Md. Suhail
Assistant Professor of English"


CONCISE SUMMARY:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary.
We have provided an existing summary up to a certain point: **Summary:**

Dr. A.P.J. Abdul Kalam's "My Vision For India" is a visionary document outlining his aspirations for India's future. Prepared by K. Md. Suhail, Assistant Professor of English at C. Abdul Hakeem College, the document focuses on the transformative power of e-learning in the Department of English, aiming to enhance educational standards and empower the country's youth.
We have the opportunity to refine the existing summary (only if needed) with some more context below.
-